# Reshaping, Reorganizing and Aggregation

In this session, we will expand our scope of data operations on DataFrame objects to include more complex techniques of manipulating data and deriving results from grouped sets of financial data. The examples in this session will focus on retrieving, organizing, reshaping, and grouping/aggregating data to be able to perform basic statistical operations.

Specifically, in this session, we will cover the following topics:

    Loading historical stock data from the Web or from files
    Concatenating and merging stock price data along multiple axes
    Merging data in multiple DataFrame objects
    Pivoting stock price data between axes
    Stacking, unstacking, and melting of stock data
    Splitting and grouping stock data to be able to calculate aggregate results

In [1]:
# import pandas, numPy, datetime functions and matplotlib 
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
# %matplotlib inline

# # Set some Pandas options
# pd.set_option('display.notebook_repr_html', False)
# pd.set_option('display.max_columns', 15)
# pd.set_option('display.max_rows', 8)
# pd.set_option('precision', 3)

## Loading historical stock data from the CSV files

The examples in this session will utilize data extracted from Yahoo! Finance. This information can be extracted live from the web services or from files provided with the session. This data consists of stock prices for MSFT and AAPL for the year 2012.

If you are not online or just want to load the data from the file, you can use the following command. I actually recommend using this data as even though the online data is historical, the adjusted close values are sometimes changed to represent other events and can potentially cause some output different than what is in the session:

In [2]:
# read the Microsoft and Apple data from file
msft = pd.read_csv("https://raw.githubusercontent.com/safarini/Python_Pandas/master/msft.csv", index_col=0, parse_dates=True)
aapl = pd.read_csv("https://raw.githubusercontent.com/safarini/Python_Pandas/master/aapl.csv", index_col=0, parse_dates=True)

### Organizing the data for the examples

With this information in hand, various slices of data are created to facilitate the various examples through the chapter, as shown here:

In [3]:
msft.head()

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2012-01-03,26.55,26.96,26.39,26.77,64731500,24.42183
2012-01-04,26.82,27.47,26.78,27.40,80516100,24.99657
2012-01-05,27.38,27.73,27.29,27.68,56081400,25.25201
2012-01-06,27.53,28.19,27.53,28.11,99455500,25.64429
2012-01-09,28.05,28.10,27.72,27.74,59706800,25.30675


In [4]:
# All columns up to the third row for MSFT
msft[:3]

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2012-01-03,26.55,26.96,26.39,26.77,64731500,24.42183
2012-01-04,26.82,27.47,26.78,27.40,80516100,24.99657
2012-01-05,27.38,27.73,27.29,27.68,56081400,25.25201


In [5]:
# All columns up to the third row AAPL
aapl[:3]

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2012-01-03,409.39996,412.50000,409.00000,411.22998,75555200,55.41362
2012-01-04,410.00000,414.68002,409.28000,413.44000,65005500,55.71143
2012-01-05,414.94998,418.55002,412.67004,418.03000,67817400,56.32993


# Reorganizing and reshaping data

When working with financial information, it is often the case that data retrieved from almost any data source will not be in the format that you need to perform the analyses that you want.

Or perhaps, just as likely, the data from a specific source may be incomplete and require collection of data from another source, at which point, the data needs to be either concatenated or merged through join-like operations across the data.

Even if the data is complete or after combining it from various sources, it may still be organized in a manner that is not conducive to a specific type of analysis. Hence, it needs to be restructured.

Fortunately, pandas provides rich capabilities for concatenating, merging, and pivoting data. These following sections take us through several common scenarios of each, using stock data.

## Concatenating data in Multiple DataFrame objects

Concatenation in pandas is the process of creating a new pandas object by combining data from two (or more pandas) objects into a new pandas object along a single, specified axis of the two objects. Concatenation with stock data is useful to combine values taken at different time periods, to create additional columns representing other measurements at a particular date and time for a specific stock, or to add a column for the same measurement of a different stock but for the same time period.

DataFrame objects are concatenated by pandas along a specified axis—the two axes being the index labels of the rows and the columns. This is done by first extracting the labels from both the DataFrame object indices along the specified axis, using that set as the index of the new DataFrame, and then copying the values along the other axis into the result in an orderly manner, that is, from the first DataFrame and then from the second DataFrame.

The result of a concatenation always contains the union of the number of items in both objects along the specific axis. As we will see later in this section, this is different than a merge or join that could result in the resulting number of items not necessarily being equivalent to the union of the number of items in the source DataFrame objects.

The tricky part of concatenation is how pandas deals with the items along the other axis during the concatenation. The set of values, be they rows when concatenating along the columns or columns when concatenating along rows, is defined using relational algebra on the values in that axis's index.

To demonstrate various forms of concatenation, we will start with the following data that shows the adjusted closing prices for MSFT for the months of January and February 2012 represented in the following command. This dataset simulates the retrieval of stock information representing two different time periods and stores the data in two different DataFrame objects, as shown here:

In [6]:
# get MSFT adjusted close data for Jan and Feb 2012
msftA01 = msft['2012-01'][['Adj Close']]
msftA02 = msft['2012-02'][['Adj Close']]
msftA01[:3]

,Adj Close
Date,
2012-01-03,24.42183
2012-01-04,24.99657
2012-01-05,25.25201


In [7]:
msftA02[:3]

,Adj Close
Date,
2012-02-01,27.26815
2012-02-02,27.32289
2012-02-03,27.58745


To combine both of these sets of data into a single DataFrame, we perform a concatenation. To demonstrate the following concatenates, the first three rows from each DataFrame are as follows:

In [8]:
# combine the first three rows of each of msftA01 and msftA02
pd.concat([msftA01.head(3), msftA02.head(3)])

,Adj Close
Date,
2012-01-03,24.42183
2012-01-04,24.99657
2012-01-05,25.25201
2012-02-01,27.26815
2012-02-02,27.32289
2012-02-03,27.58745


The resulting DataFrame contains an index identical in structure to both of the objects, with labels from the first object and then the second object copied into the new object. At first glance, it may appear that the concatenation is a pure copy of the rows from each DataFrame into the new DataFrame, but as we will see, the process is more elaborate (and hence flexible). This will become more evident as we take a look at more examples.

The following example concatenates the first five adjusted close values in January for both MSFT and AAPL. These have identical index labels and result in duplicate index labels in the new DataFrame. During a concatenation along the row axis, pandas will not align the index labels. They will be copied and this can create duplicate, identical index labels:

In [9]:
# Extract only the Jan 2012 AAPL values.  
aaplA01 = aapl['2012-01'][['Adj Close']]
# now concat the AAPL and MSFT Jan 2012 data
# there will be duplicate index labels
withDups = pd.concat([msftA01[:3], aaplA01[:3]])
withDups

,Adj Close
Date,
2012-01-03,24.42183
2012-01-04,24.99657
2012-01-05,25.25201
2012-01-03,55.41362
2012-01-04,55.71143
2012-01-05,56.32993


This has resulted in duplicated index labels and will result in multiple items being returned for those labels, as shown here:

In [10]:
# show the two records for data of 2012-01-03
withDups.loc['2012-01-03']

,Adj Close
Date,
2012-01-03,24.42183
2012-01-03,55.41362


This concatenation has lost whether the Adj Close value in the new DataFrame came from the MSFT or AAPL DataFrame. This source DataFrame of each row can be preserved during concatenation by specifying the value of the keys in the new DataFrame. These keys will add an additional level to the index (making a MultiIndex), which then can be used to identify the source DataFrame:

In [11]:
# demonstrate concat with a specification of the 
# stock tickets being part of the index
# this help disambiguate the duplicate dates using
# a hierarchal index
closes = pd.concat([msftA01[:3], aaplA01[:3]], 
                    keys=['MSFT', 'AAPL'])
closes

Adj Close
     Date                 
MSFT 2012-01-03   24.42183
     2012-01-04   24.99657
     2012-01-05   25.25201
AAPL 2012-01-03   55.41362
     2012-01-04   55.71143
     2012-01-05   56.32993

Using this new MultiIndex, it is then possible to extract the values for either stock from this new DataFrame by only using the index labels. The following command does this for the MSFT entries:

In [12]:
# extract just MSFT values using .loc
closes.loc['MSFT'][:3]

,Adj Close
Date,
2012-01-03,24.42183
2012-01-04,24.99657
2012-01-05,25.25201


In [13]:
# demonstrate concatenation using two DataFrame's
# that each have two columns.  pandas will align the
# data in columns by the column names (labels)
msftAV = msft[['Adj Close', 'Volume']]
aaplAV = aapl[['Adj Close', 'Volume']]
pd.concat([msftAV, aaplAV])

,Adj Close,Volume
Date,,
2012-01-03,24.42183,64731500
2012-01-04,24.99657,80516100
2012-01-05,25.25201,56081400
2012-01-06,25.64429,99455500
2012-01-09,25.30675,59706800
...,...,...
2012-12-21,70.60166,149067100
2012-12-24,70.71585,43938300
2012-12-26,69.74111,75609100


The columns in the DataFrame objects in a concatenation do not have to have the same names. The following command demonstrates a concatenation where the aaplA DataFrame only consists of the Adj Close column, whereas the MSFT DataFrame has both Adj Close and Volume columns:

In [14]:
# demonstrate concatenation with DataFrame objects
# that do not have the same set of columns
# this demonstrates pandas filling in NaN values
aaplA = aapl[['Adj Close']]
pd.concat([msftAV, aaplA])

,Adj Close,Volume
Date,,
2012-01-03,24.42183,64731500.0
2012-01-04,24.99657,80516100.0
2012-01-05,25.25201,56081400.0
2012-01-06,25.64429,99455500.0
2012-01-09,25.30675,59706800.0
...,...,...
2012-12-21,70.60166,NaN
2012-12-24,70.71585,NaN
2012-12-26,69.74111,NaN


Since the rows originating from the aapl DataFrame do not have a Volume column, pandas inserts NaN into the Volume column for those rows.

The set of columns that results from a concatenation along the row axis is the result of relational algebra across the names of the columns. In this default scenario, the resulting column is the union of column names from each DataFrame. This can be changed to an intersection using the join parameter. The following command makes the set of resulting columns the intersection of the column names by specifying join='inner':

In [15]:
# perform an inner join on the DataFrame's
# since aaplA does not have a Volume column, pandas
# will not include that column in the result
pd.concat([msftAV, aaplA], join='inner')

,Adj Close
Date,
2012-01-03,24.42183
2012-01-04,24.99657
2012-01-05,25.25201
2012-01-06,25.64429
2012-01-09,25.30675
...,...
2012-12-21,70.60166
2012-12-24,70.71585
2012-12-26,69.74111


We can change the axis for concatenation to the columns using axis=1:

In [16]:
# concat along the rows, causing duplicate columns to
# be created in the result
msftA = msft[['Adj Close']]
closes = pd.concat([msftA, aaplA], axis=1)
closes[:3]

,Adj Close,Adj Close
Date,,
2012-01-03,24.42183,55.41362
2012-01-04,24.99657,55.71143
2012-01-05,25.25201,56.32993


Note that this DataFrame has two Adj Close columns and only consists of 249 rows (the concatenation along axis=0 has 498). Because of the use of axis=1, the union of the index labels is derived instead from the column names, and the columns are copied one by one in an orderly manner from the DataFrame objects, including duplicates.

It is also possible to concatenate with multiple columns where the DataFrame objects do not have the same set of index labels. The following command concatenates the first five msftAV values and the first three aaplAV values:

In [17]:
# concat along rows using two DataFrame objects with
# different number of rows. This demonstrates how
# NaN values will be filled in those rows for AAPL
# which only hase three rows as compared to 5 for MSFT
pd.concat([msftAV[:5], aaplAV[:3]], axis=1,
          keys=['MSFT', 'AAPL'])

MSFT                AAPL            
           Adj Close    Volume Adj Close      Volume
Date                                                
2012-01-03  24.42183  64731500  55.41362  75555200.0
2012-01-04  24.99657  80516100  55.71143  65005500.0
2012-01-05  25.25201  56081400  56.32993  67817400.0
2012-01-06  25.64429  99455500       NaN         NaN
2012-01-09  25.30675  59706800       NaN         NaN

This results in duplicate column names, so we use the keys parameter to create MultiIndex for the columns. Since there were row index labels that were not found in aaplCV, pandas fills those with NaN.

Just as with concatenation along the row axis, the type of join performed by pd.concat() can be changed using the join parameter. The following command performs an inner join instead of an outer join, which results in the intersection of row index labels:

In [18]:
# inner join can also be used along this axis
# this will not include rows with index labels that do
# not exist in both DataFrame objects
pd.concat([msftA[:5], aaplA[:3]], axis=1,
          join='inner', keys=['MSFT', 'AAPL'])

,MSFT,AAPL
,Adj Close,Adj Close
Date,,
2012-01-03,24.42183,55.41362
2012-01-04,24.99657,55.71143
2012-01-05,25.25201,56.32993


The resulting DataFrame only has three rows because those index labels were the only ones in common in the two concatenated DataFrame objects.

If you want to ignore indices in the result of pd.concat(), you can use the ignore_index=True parameter, which will drop the index and create a default zero-based integer index, as shown here:

In [19]:
# ignore indexes and just concatenate the data and
# have the result have a default integer index
pd.concat([msftA[:3], aaplA[:3]], ignore_index=True)

,Adj Close
0,24.42183
1,24.99657
2,25.25201
3,55.41362
4,55.71143
5,56.32993


## Merging DataFrame objects

The combination of pandas objects is allowed using relational database-like join operations, high-performance in-memory operations, and the pd.merge() function.

Merging in pandas differs from concatenation in that the pd.merge() function combines data based on the values of the data in one or more columns instead of using the index label values along a specific axis.

The default process that pd.merge() uses is to first identify the columns the data of which will be used in the merge, and then to perform an inner join based upon that information. The columns used in the join are, by default, selected as those in both DataFrame objects with common names (an intersection of the column labels).

To demonstrate a merge, we will use the following two DataFrame objects, one with the volumes and the other with the adjusted close values for MSFT. Both have the index reset:

In [20]:
# we will merge these two DataFrame objects, 
# so lets peek at the data to remind ourselves
# of what they contain
msftAR = msftA.reset_index()
msftVR = msft[['Volume']].reset_index()
msftAR[:3]

,Date,Adj Close
0,2012-01-03,24.42183
1,2012-01-04,24.99657
2,2012-01-05,25.25201


In [21]:
msftVR[:3]

,Date,Volume
0,2012-01-03,64731500
1,2012-01-04,80516100
2,2012-01-05,56081400


Instead of using Date as the index, these have Date as a column so that it can be used in the merge. Our goal is to create a DataFrame that contains a Date column and both AdjClose and Volume columns. This can be accomplished with the following statement:

In [22]:
# merge the two.  pandas finds the columns in common,
# in this case Date, and merges on that column and adds
# a column for all the other columns in both DataFrame's
msftCVR = pd.merge(msftAR, msftVR)
msftCVR[:5]

,Date,Adj Close,Volume
0,2012-01-03,24.42183,64731500
1,2012-01-04,24.99657,80516100
2,2012-01-05,25.25201,56081400
3,2012-01-06,25.64429,99455500
4,2012-01-09,25.30675,59706800


The column in common is Date; therefore, pandas performs an inner join on the values in that column across both DataFrame objects. Once that set is calculated, pandas copies in the appropriate values for each row from both DataFrame objects.

The types of joins supported by pd.merge() are similar to the different types of joins supported in relational databases. They are as follows:

left: Use keys from the left DataFrame (equivalent to SQL's left-outer join)
right: Use keys from the right DataFrame (equivalent to SQL's right-outer join)
outer: Use the union of keys from both DataFrame objects (equivalent to SQL's full outer join)
inner: Use the intersection of keys from both DataFrame objects (equivalent to SQL's inner join)
To demonstrate each difference in the results between inner and outer joins, we will use the following data:

In [23]:
# we will demonstrate join semantics using this DataFrame
msftAR0_5 = msftAR[0:5]
msftAR0_5

,Date,Adj Close
0,2012-01-03,24.42183
1,2012-01-04,24.99657
2,2012-01-05,25.25201
3,2012-01-06,25.64429
4,2012-01-09,25.30675


In [24]:
# and also this one
msftVR2_4 = msftVR[2:4]
msftVR2_4

,Date,Volume
2,2012-01-05,56081400
3,2012-01-06,99455500


For an inner join, since there are only two rows with matching dates, the result only has two rows and merges the DataFrame objects where Date values are in common, as shown here:

In [25]:
# merge semantics using default inner join
pd.merge(msftAR0_5, msftVR2_4)

,Date,Adj Close,Volume
0,2012-01-05,25.25201,56081400
1,2012-01-06,25.64429,99455500


This can be changed to an outer join with how='outer'. All rows from the outer DataFrame are returned (msftAR0_5), and values not found in the inner DataFrame (msftVR2_4) are replaced with NaN:

In [26]:
# same joing but using
pd.merge(msftAR0_5, msftVR2_4, how='outer')

,Date,Adj Close,Volume
0,2012-01-03,24.42183,NaN
1,2012-01-04,24.99657,NaN
2,2012-01-05,25.25201,56081400.0
3,2012-01-06,25.64429,99455500.0
4,2012-01-09,25.30675,NaN


## Pivoting

Financial data is often stored in a format where the data is not normalized and, therefore, has repeated values in many columns or values that logically should exist in other tables. An example of this would be the following, where the historical prices for multiple stocks are represented in a single DataFrame using a Symbol column. The following command creates a DataFrame with this schema and populates the records:



In [27]:
# need to insert Symbol column before combining
msft.insert(0, 'Symbol', 'MSFT')
aapl.insert(0, 'Symbol', 'AAPL')

# concatenate the MSFT and AAPL data
# index will consist of the Date column, which we will sort
combined = pd.concat([msft, aapl]).sort_index()

# this pushes the index into a column and resets to a 
# default integer index
s4p = combined.reset_index();
s4p[:5]

,Date,Symbol,Open,High,Low,Close,Volume,Adj Close
0,2012-01-03,MSFT,26.55000,26.96000,26.39,26.77000,64731500,24.42183
1,2012-01-03,AAPL,409.39996,412.50000,409.00,411.22998,75555200,55.41362
2,2012-01-04,MSFT,26.82000,27.47000,26.78,27.40000,80516100,24.99657
3,2012-01-04,AAPL,410.00000,414.68002,409.28,413.44000,65005500,55.71143
4,2012-01-05,MSFT,27.38000,27.73000,27.29,27.68000,56081400,25.25201


Now let's suppose we want to extract, from this DataFrame, a new DataFrame that is indexed by date and has columns representing the AdjClose value for all of the stocks listed in the Symbol column. This can be performed using the .pivot() method of the DataFrame:

In [28]:
# pivot Date into the Index, make the columns match the
# unique values in the Symbol column, and the values 
# will be the AdjClose values
closes = s4p.pivot(index='Date', columns='Symbol', 
                   values='Adj Close')
closes[:3]

Symbol,AAPL,MSFT
Date,,
2012-01-03,55.41362,24.42183
2012-01-04,55.71143,24.99657
2012-01-05,56.32993,25.25201


This has taken all the distinct values from the Symbol column, pivoted them into columns on the new DataFrame, and then entered the values in those columns from the AdjClose value for the specific symbol from the original DataFrame.

## Stacking and Unstacking

The DataFrame methods similar in operation to the pivot function are .stack() and .unstack(). Stacking unpivots the column labels into another level of the index. To demonstrate this, the following command pivots the MSFT and AAPL columns into the index:

In [29]:
# stack the first level of columns into the index
# essentially, moves AAPL and MSFT into the index
# leaving a single colum which is the AdjClose values
stackedCloses = closes.stack()
stackedCloses

Date        Symbol
2012-01-03  AAPL      55.41362
            MSFT      24.42183
2012-01-04  AAPL      55.71143
            MSFT      24.99657
2012-01-05  AAPL      56.32993
                        ...   
2012-12-26  MSFT      25.19693
2012-12-27  AAPL      70.02116
            MSFT      25.29074
2012-12-28  AAPL      69.27753
            MSFT      24.90612
Length: 498, dtype: float64

This has created a new index with an additional level named Symbol (from the name of the columns index). Each row is then indexed by both Date and Symbol. And for each unique Date and Symbol level, pandas has inserted the appropriate Adj Close value.

The result of this allows the efficient lookup of any Adj Close value using the index. To look up the Adj Close value for AAPL on 2012-01-03, we can use the following command:

In [30]:
# using .ix we can retrieve close values by
# specifying both the date and ticker
stackedCloses.loc['2012-01-03', 'AAPL']

55.41362

The result here is equivalent to the following value-based lookup, but is significantly more efficient, uses less typing, and is also organized better, causing less mental clutter.

Using a MultiIndex, it is also possible to look up values for just a specific Date:

In [31]:
# lookup on just the date, which will give us two values
# one each for AAPL and MSFT.  
stackedCloses.loc['2012-01-03']

Date        Symbol
2012-01-03  AAPL      55.41362
            MSFT      24.42183
dtype: float64

For a specific Symbol, here is the command:

In [32]:
# this looks up all values for the MSFT symbol
stackedCloses.loc[:, 'MSFT']

Date
2012-01-03    24.42183
2012-01-04    24.99657
2012-01-05    25.25201
2012-01-06    25.64429
2012-01-09    25.30675
                ...   
2012-12-21    25.75040
2012-12-24    25.38455
2012-12-26    25.19693
2012-12-27    25.29074
2012-12-28    24.90612
Length: 249, dtype: float64

The .unstack() method performs the opposite function; that is, it pivots a level of an index into a column in a new DataFrame. The following command unstacks the last level of the MultiIndex and results in a DataFrame equivalent to the original unstackedCloses:

In [33]:
# pivots the last level of the index back into a column
unstackedCloses = stackedCloses.unstack()
unstackedCloses[:3]

Symbol,AAPL,MSFT
Date,,
2012-01-03,55.41362,24.42183
2012-01-04,55.71143,24.99657
2012-01-05,56.32993,25.25201


## Melting

Melting is the process of transforming a DataFrame into a format where each row represents a unique id-variable combination. The following command demonstrates melting the s4p DataFrame into an id-variable combination consisting of the Date and Symbol columns as the ID and the other columns mapped into the variables:

In [34]:
# melt making id_vars of Date and Symbol, making the 
# column names the variable and the for each the value
melted = pd.melt(s4p, id_vars=['Date', 'Symbol'])
melted[:5]

,Date,Symbol,variable,value
0,2012-01-03,MSFT,Open,26.55000
1,2012-01-03,AAPL,Open,409.39996
2,2012-01-04,MSFT,Open,26.82000
3,2012-01-04,AAPL,Open,410.00000
4,2012-01-05,MSFT,Open,27.38000


During a melt, the column(s) specified by the id_vars parameter remain columns (in this case Date and Symbol). All other columns have their names mapped to the values in the variable column—one row for each variable column of an id_var column value combination.

This organization of data is useful to select chunks of information based upon a specific ID variable and then one or more variables. As an example, the following command returns all measurements for 2012-01-03 and the MSFT symbol:

In [35]:
# extract the values for the data for MSFT on 2012-01-03
melted[(melted.Date=='2012-01-03') & (melted.Symbol=='MSFT')]

,Date,Symbol,variable,value
0,2012-01-03,MSFT,Open,2.655000e+01
498,2012-01-03,MSFT,High,2.696000e+01
996,2012-01-03,MSFT,Low,2.639000e+01
1494,2012-01-03,MSFT,Close,2.677000e+01
1992,2012-01-03,MSFT,Volume,6.473150e+07
2490,2012-01-03,MSFT,Adj Close,2.442183e+01


# Grouping and aggregation

Data in pandas can be easily split into groups and then summarized using various statistical and quantitative calculations. This process in pandas nomenclature is often referred to as the split-apply-combine pattern.

In this section, we will look at using this pattern as applied to stock data. We will split the data by various time and symbol combinations and then apply statistical operations to begin analyzing the risk and return on our sample data.

## Splitting

Objects in pandas are split into groups using the .groupby() method. To demonstrate this, we will use the stock price data introduced earlier in the chapter but slightly reorganized to facilitate understanding of the grouping process:

In [36]:
# construct a DataFrame to demonstrate splitting
# extract from combined the Symbol and AdjClose, and reset the index
s4g = combined[['Symbol', 'Adj Close']].reset_index()
# now, add two columns, year and month, using the year and month
# portions of the data as integers
s4g.insert(1, 'Year', pd.DatetimeIndex(s4g['Date']).year)
s4g.insert(2, 'Month',pd.DatetimeIndex(s4g['Date']).month)
s4g[:5]

,Date,Year,Month,Symbol,Adj Close
0,2012-01-03,2012,1,MSFT,24.42183
1,2012-01-03,2012,1,AAPL,55.41362
2,2012-01-04,2012,1,MSFT,24.99657
3,2012-01-04,2012,1,AAPL,55.71143
4,2012-01-05,2012,1,MSFT,25.25201


This data differs from before as only the AdjClose value is utilized, and the Date column is broken apart into two other columns, Year and Month. This splitting of the date is done to be able to provide the ability to group the data by Month and Year for each Symbol variable.

This data consists of four categorical variables (Date, Symbol, Year, and Month) and one continuous variable, AdjClose. In pandas, it is possible to group by any single categorical variable by passing its name to .groupby(). The following command groups by the Symbol column:

In [37]:
# group by the Symbol column
s4g.groupby('Symbol')

The result of calling .groupby() on a DataFrame is not the actual grouped data but a DataFrameGroupBy object (a SeriesGroupBy for a grouping on a Series). The grouping has not actually been performed yet as grouping is a deferred/lazy process in pandas.

This result of .groupby() is a subclass of a GroupBy object and is an interim description of the grouping to be performed (if you are a C# programmer, this feels a lot like an expression tree created by LINQ). This allows pandas to first validate that the grouping description provided to it is valid relative to the data before the processing starts.

There are number of useful properties on a GroupBy object. The .groups property will return a Python dictionary whose keys represent the name of each group (if multiple columns are specified, it is a tuple), and the values are an array of the index labels contained within each group:

In [38]:
# group again, but save the result this time
grouped = s4g.groupby('Symbol')
# the groupby object has a property groups, which shows how
# all rows will in mapped into the groups.   
# the type of this object is a python dict
type(grouped.groups)

dict

In [39]:
# show the mappings of rows to groups
grouped.groups

{'AAPL': Int64Index([  1,   3,   5,   7,   9,  11,  13,  14,  16,  18,
             ...
             478, 480, 482, 484, 487, 488, 490, 492, 494, 497],
            dtype='int64', length=249),
 'MSFT': Int64Index([  0,   2,   4,   6,   8,  10,  12,  15,  17,  19,
             ...
             479, 481, 483, 485, 486, 489, 491, 493, 495, 496],
            dtype='int64', length=249)}

The Python len() function can be used to return the number of groups, which will result from the grouping as well as the .ngroups property:

In [40]:
# these report the number of groups that resulted from
# the grouping
len(grouped), grouped.ngroups

(2, 2)

Splitting is not performed until you take some type of action on the GroupBy object. It is, however, possible to iterate over several properties of the object to view how the data will be grouped (hence forcing it to be grouped). The following helper function demonstrates this and will be used frequently throughout this chapter:

In [41]:
# this function will print the contents of a group
def print_groups (groupobject):
    for name, group in groupobject:
        print (name)
        print (group.head())

In [42]:
# examine our resulting groups
print_groups(grouped)

AAPL
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL   55.41362
3 2012-01-04  2012      1   AAPL   55.71143
5 2012-01-05  2012      1   AAPL   56.32993
7 2012-01-06  2012      1   AAPL   56.91880
9 2012-01-09  2012      1   AAPL   56.82851
MSFT
        Date  Year  Month Symbol  Adj Close
0 2012-01-03  2012      1   MSFT   24.42183
2 2012-01-04  2012      1   MSFT   24.99657
4 2012-01-05  2012      1   MSFT   25.25201
6 2012-01-06  2012      1   MSFT   25.64429
8 2012-01-09  2012      1   MSFT   25.30675


Looking at these results gives us some insight into what pandas is doing with this specific splitting operation. It has created, for each distinct value in the Symbol column of the original DataFrame, a group consisting of a DataFrame (this is different from the functionality provided by itertools.groupby, so be careful if you are used to using that library for this functionality). It then copies the non-grouped columns and data into each of those DataFrame objects and then uses the values from the specified column(s) as the group name.

The .size() method of the object gives a nice summary of the size of all the groups:

In [43]:
# .size will tell us the count of items in each group
grouped.size()

Symbol
AAPL    249
MSFT    249
dtype: int64

If you want the data for the items in any given group, you can use the .get_group() property. The following command retrieves the MSFT group:

In [44]:
# a specific group can be retrieved using .get_group()
# which returns a DataFrame representing the specified group
grouped.get_group('MSFT')

,Date,Year,Month,Symbol,Adj Close
0,2012-01-03,2012,1,MSFT,24.42183
2,2012-01-04,2012,1,MSFT,24.99657
4,2012-01-05,2012,1,MSFT,25.25201
6,2012-01-06,2012,1,MSFT,25.64429
8,2012-01-09,2012,1,MSFT,25.30675
...,...,...,...,...,...
489,2012-12-21,2012,12,MSFT,25.75040
491,2012-12-24,2012,12,MSFT,25.38455
493,2012-12-26,2012,12,MSFT,25.19693
495,2012-12-27,2012,12,MSFT,25.29074


Grouping can be performed upon multiple columns by passing a list of column names. The following command groups the data by the Symbol and Year and Month variables:

In [45]:
# group by three different fields and print the result
mcg = s4g.groupby(['Symbol', 'Year', 'Month'])
print_groups(mcg)

('AAPL', 2012, 1)
        Date  Year  Month Symbol  Adj Close
1 2012-01-03  2012      1   AAPL   55.41362
3 2012-01-04  2012      1   AAPL   55.71143
5 2012-01-05  2012      1   AAPL   56.32993
7 2012-01-06  2012      1   AAPL   56.91880
9 2012-01-09  2012      1   AAPL   56.82851
('AAPL', 2012, 2)
         Date  Year  Month Symbol  Adj Close
41 2012-02-01  2012      2   AAPL   61.47202
43 2012-02-02  2012      2   AAPL   61.32784
44 2012-02-03  2012      2   AAPL   61.94231
46 2012-02-06  2012      2   AAPL   62.52039
49 2012-02-07  2012      2   AAPL   63.17527
('AAPL', 2012, 3)
         Date  Year  Month Symbol  Adj Close
81 2012-03-01  2012      3   AAPL   73.36784
82 2012-03-02  2012      3   AAPL   73.46351
84 2012-03-05  2012      3   AAPL   71.84380
86 2012-03-06  2012      3   AAPL   71.45303
89 2012-03-07  2012      3   AAPL   71.51097
('AAPL', 2012, 4)
          Date  Year  Month Symbol  Adj Close
124 2012-04-02  2012      4   AAPL   83.36096
126 2012-04-03  2012      4   AA

Since multiple columns were specified, the name of each group is now a tuple with the value from Symbol, Year, and Month that represents the group.

The examples up to this point have used a DataFrame without any specific indexing (just the default sequential numerical index). This type of data would actually be very well suited for a hierarchical index, which can then be used directly to group the data based upon index label(s). To demonstrate this, the following command creates a new DataFrame with a MultiIndex consisting of the original Symbol, Year, and Month columns:

In [46]:
# set the index of the data to be the following three fields
# we are creating a multiindex
mi = s4g.set_index(['Symbol', 'Year', 'Month'])
mi

Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03   24.42183
AAPL   2012 1     2012-01-03   55.41362
MSFT   2012 1     2012-01-04   24.99657
AAPL   2012 1     2012-01-04   55.71143
MSFT   2012 1     2012-01-05   25.25201
...                      ...        ...
            12    2012-12-26   25.19693
AAPL   2012 12    2012-12-27   70.02116
MSFT   2012 12    2012-12-27   25.29074
            12    2012-12-28   24.90612
AAPL   2012 12    2012-12-28   69.27753

[498 rows x 2 columns]

Grouping can now be performed using the levels of the hierarchical index. The following groups by the index level 0 (Symbol names):

In [47]:
# now we can group based upon values in the actual index
# the following groups by level 0 of the index (Month)
mig_l1 = mi.groupby(level=0)
print_groups(mig_l1)

AAPL
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03   55.41362
            1     2012-01-04   55.71143
            1     2012-01-05   56.32993
            1     2012-01-06   56.91880
            1     2012-01-09   56.82851
MSFT
                        Date  Adj Close
Symbol Year Month                      
MSFT   2012 1     2012-01-03   24.42183
            1     2012-01-04   24.99657
            1     2012-01-05   25.25201
            1     2012-01-06   25.64429
            1     2012-01-09   25.30675


Grouping by multiple levels can be performed by passing multiple levels to .groupby(). Also, if the MultiIndex has names specified, then those names can be used instead of the integers for the levels. The following command groups the three levels of the MultiIndex by using their names:

In [48]:
# group by three levels in the index using their names
mig_l12 = mi.groupby(level=['Symbol', 'Year', 'Month'])
print_groups(mig_l12)

('AAPL', 2012, 1)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 1     2012-01-03   55.41362
            1     2012-01-04   55.71143
            1     2012-01-05   56.32993
            1     2012-01-06   56.91880
            1     2012-01-09   56.82851
('AAPL', 2012, 2)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 2     2012-02-01   61.47202
            2     2012-02-02   61.32784
            2     2012-02-03   61.94231
            2     2012-02-06   62.52039
            2     2012-02-07   63.17527
('AAPL', 2012, 3)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2012 3     2012-03-01   73.36784
            3     2012-03-02   73.46351
            3     2012-03-05   71.84380
            3     2012-03-06   71.45303
            3     2012-03-07   71.51097
('AAPL', 2012, 4)
                        Date  Adj Close
Symbol Year Month                      
AAPL   2

# Aggregation

Armed with the capability to group the stock data on a monthly basis, we can now start to drive analysis of the data. Specifically, we will develop methods to calculate the risk on a stock based on a time-window of a calendar month.

Aggregation is performed using the .aggregate(), or in short .agg(), method of the GroupBy object. The parameter set to .agg() is a reference to a function that is applied to each group. The following command will calculate the mean of the values across each Symbol, Year, and Month:

In [49]:
# this will apply the mean function to each group
mig_l12.agg(np.mean)

Adj Close
Symbol Year Month           
AAPL   2012 1      57.751284
            2      67.048153
            3      77.819703
            4      81.659466
            5      76.090236
            6      77.422814
            7      80.994484
            8      86.882818
            9      92.236305
            10     85.895557
            11     76.652538
            12     72.330769
MSFT   2012 1      26.015133
            2      28.224246
            3      29.564324
            4      28.931165
            5      27.853488
            6      27.434106
            7      27.504969
            8      28.173740
            9      28.640845
            10     27.044293
            11     26.004902
            12     25.309006

The result of the aggregation will have an identically structured index as the original data. If you do not want this to happen, you can use the as_index=False option of the .groupby() method to specify not to duplicate the structure of the index, which may be convenient in several situations, including where a function expects the data with a numerical index:

In [50]:
# example of groupby that also ignores the index
# resulting in a default integer index
# this also has the mean function applied
s4g.groupby(['Symbol', 'Year', 'Month'], 
            as_index=False).agg(np.mean)[:5]

,Symbol,Year,Month,Adj Close
0,AAPL,2012,1,57.751284
1,AAPL,2012,2,67.048153
2,AAPL,2012,3,77.819703
3,AAPL,2012,4,81.659466
4,AAPL,2012,5,76.090236


This has derived the same results, but there is a slightly different organization.

It is possible to apply multiple aggregation functions to each group in a single call to .agg() by passing them in a list:

In [51]:
# apply multiple functions to each group in one call
mig_l12.agg([np.mean, np.std])

Adj Close          
                        mean       std
Symbol Year Month                     
AAPL   2012 1      57.751284  1.797870
            2      67.048153  3.569969
            3      77.819703  4.162433
            4      81.659466  3.062644
            5      76.090236  1.991461
            6      77.422814  1.012879
            7      80.994484  1.495667
            8      86.882818  3.308121
            9      92.236305  1.930281
            10     85.895557  2.868979
            11     76.652538  2.982066
            12     72.330769  2.783399
MSFT   2012 1      26.015133  0.833578
            2      28.224246  0.649867
            3      29.564324  0.311996
            4      28.931165  0.577378
            5      27.853488  0.782678
            6      27.434106  0.682946
            7      27.504969  0.579592
            8      28.173740  0.472539
            9      28.640845  0.430641
            10     27.044293  0.666642
            11     26.004902  1.001457
            12     25.309006  0.364051

In this session, we examined several fundamental techniques for loading (importing and reading data), combining, grouping, and analyzing stock pricing data with pandas. In the next chapter on time-series data with pandas, we will dive deeper into working with data provided in different time frequencies, converting the periods of data into other frequencies, and working with aggregating data based upon sliding/rolling windows instead of simple calendar months.